In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

sys.path.append(str(Path().cwd().parent.resolve()))

from pprint import PrettyPrinter
pp = PrettyPrinter()

# Uncomment to get more debugging printouts:
"""
import logging

root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)
"""

"\nimport logging\n\nroot = logging.getLogger()\nroot.setLevel(logging.DEBUG)\n\nhandler = logging.StreamHandler(sys.stdout)\nhandler.setLevel(logging.DEBUG)\nformatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')\nhandler.setFormatter(formatter)\nroot.addHandler(handler)\n"

In [2]:
import openai
def wrap_request(self, *args, **kwargs):
    print("wrapping request=")
    pp.pprint(args)
    pp.pprint(kwargs)
    rets = openai.Completion.request(self, *args, **kwargs)
    print("rets=")
    pp.pprint(rets)
    return rets

openai.Completion.request = wrap_request

help(openai.Completion.request)

Help on function wrap_request in module __main__:

wrap_request(self, *args, **kwargs)



In [3]:
from trulens_eval.keys import *

KEY SET: OPENAI_API_KEY
KEY SET: PINECONE_API_KEY
KEY SET: PINECONE_ENV
KEY SET: HUGGINGFACE_API_KEY
KEY SET: SLACK_TOKEN
KEY SET: SLACK_SIGNING_SECRET
KEY SET: COHERE_API_KEY


In [4]:
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('llama_index/data').load_data()
index = GPTVectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine()
# response = query_engine.query("What did the author do growing up?")
# print(response)

In [5]:
# For aggregation,
import numpy as np

from trulens_eval import feedback, Feedback, Query, Tru

# Construct feedback functions.

hugs = feedback.Huggingface()
openai = feedback.OpenAI()

# Language match between question/answer.
f_lang_match = Feedback(hugs.language_match).on(
    text1=Query.RecordInput, text2=Query.RecordOutput
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(openai.relevance).on(
    prompt=Query.RecordInput, response=Query.RecordOutput
)

# Question/statement relevance between question and each context chunk.
f_qs_relevance = feedback.Feedback(openai.qs_relevance).on(
    question=Query.RecordInput,
    statement=Query.Record.model.retriever.retrieve.rets[:].node.text
).aggregate(np.min)

feedbacks = [
#    f_lang_match, 
#    f_qa_relevance, 
#    f_qs_relevance
]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface api: 0requests [00:00, ?requests/s]

openai api: 0requests [00:00, ?requests/s]

In [6]:
l = Tru().Llama(engine=query_engine, feedbacks=feedbacks, chain_id="default")

✅ app app_hash_fd4bc0b7a6be4243930b26606ad25b6f -> default.sqlite


In [7]:
list(l.instrumented())

[(JSONPath().app, <trulens_eval.utils.llama.Other at 0x1546128b0>),
 (JSONPath().app._response_synthesizer,
  <trulens_eval.utils.llama.Other at 0x15460bfa0>),
 (JSONPath().app._response_synthesizer._response_builder,
  <trulens_eval.utils.llama.Other at 0x1547af0a0>),
 (JSONPath().app._response_synthesizer._response_builder._refine_template,
  <trulens_eval.utils.llama.Prompt at 0x1547afdc0>),
 (JSONPath().app._response_synthesizer._response_builder._refine_template.prompt,
  <trulens_eval.utils.langchain.Prompt at 0x1547d1580>),
 (JSONPath().app._response_synthesizer._response_builder.service_context,
  <trulens_eval.utils.llama.Other at 0x1547d1220>),
 (JSONPath().app._response_synthesizer._response_builder.service_context.embed_model,
  <trulens_eval.utils.llama.Other at 0x1547d14f0>),
 (JSONPath().app._response_synthesizer._response_builder.service_context.llm_predictor,
  <trulens_eval.utils.llama.Other at 0x1547d1c10>),
 (JSONPath().app._response_synthesizer._response_builder.se

In [8]:
res, record = l.query_with_record("Who is Shayak?")

Token indices sequence length is longer than the specified maximum sequence length for this model (1818 > 1024). Running this sequence through the model will result in indexing errors


✅ record record_hash_bc51cb49c6f48294717659eb66396bbe from app_hash_fd4bc0b7a6be4243930b26606ad25b6f -> default.sqlite


In [ ]:
proc = Tru().start_dashboard(force=True, _dev=Path.cwd().parent)
# thread = Tru().start_evaluator(restart=True)

In [ ]:
import llama_index
dir(llama_index.llm_predictor)

In [ ]:
type(l.app._response_synthesizer._response_builder)